# **🍀Qwen 8B - QLoRA** 

## **1. 라이브러리 호출**

In [1]:
import os
from peft import PeftModel
import json
import torch
import glob
from sklearn.model_selection import train_test_split
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, Trainer, DataCollatorForLanguageModeling
from trl import SFTTrainer
from model_load import tokenizer, model

/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [01:36<00:00, 19.21s/it]


## **2. 데이터 준비**

In [2]:
def build_context_dict(docs):
    context_dict = {}
    for doc in docs:
        source_id = doc["metadata"].get("source_id")
        chunk_id = doc["metadata"].get("chunk_id")
        if source_id and chunk_id:
            context_dict[(source_id, chunk_id)] = doc["text"]
    return context_dict

In [3]:
def convert_to_sft_format(qa_path, context_dict, output_path):
    with open(qa_path, "r", encoding="utf-8") as f_in, \
         open(output_path, "w", encoding="utf-8") as f_out:
        for line in f_in:
            qa = json.loads(line.strip())
            key = (qa["source_id"], qa["chunk_id"])
            context = context_dict.get(key, "")
            sft_item = {
                "input": f"문서 내용:\n{context}\n\n질문:\n{qa['Question']}",
                "output": qa["Answer"]
            }
            f_out.write(json.dumps(sft_item, ensure_ascii=False) + "\n")

In [4]:
from glob import glob 

In [5]:
# 원본 문서 불러오기
dummy_dir = "/home/NEUL77/AI-Engineer/data/dummy"
jsonl_files = glob(os.path.join(dummy_dir, "*.jsonl"))

docs = []
for file_path in jsonl_files:
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            docs.append({
                "text": data["text"],
                "metadata": data.get("metadata", {})
            })

# source_id + chunk_id 기반 context dict 생성
context_dict = build_context_dict(docs)

# QA + context → SFT 포맷 변환
qa_path = "/home/NEUL77/AI-Engineer/data/train/qa_dataset.jsonl"
output_path = "/home/NEUL77/AI-Engineer/data/train/qa_sft_dataset.jsonl"
convert_to_sft_format(qa_path, context_dict, output_path)

In [6]:
def split_input_fields(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as f_in, \
         open(output_path, "w", encoding="utf-8") as f_out:

        for line in f_in:
            item = json.loads(line.strip())

            full_input = item.get("input", "")
            output = item.get("output", "")

            # "문서 내용:"과 "질문:" 기준으로 분리
            try:
                doc_part, question_part = full_input.split("질문:")
                doc_text = doc_part.replace("문서 내용:", "").strip()
                question_text = question_part.strip()

                new_item = {
                    "instruction": question_text,  # 질문
                    "input": doc_text,             # 문서 내용
                    "output": output               # 정답
                }
                f_out.write(json.dumps(new_item, ensure_ascii=False) + "\n")

            except ValueError:
                print("⚠️ 잘못된 형식의 input 필드가 있습니다. 건너뜁니다.")
                continue

    print(f"[✔] 분리 완료: {output_path}")

In [7]:
# 경로 설정
input_path = "/home/NEUL77/AI-Engineer/data/train/qa_sft_dataset.jsonl"
output_path = "/home/NEUL77/AI-Engineer/data/train/qa_sft_dataset_split.jsonl"

# 실행
split_input_fields(input_path, output_path)

[✔] 분리 완료: /home/NEUL77/AI-Engineer/data/train/qa_sft_dataset_split.jsonl


In [8]:
def data_loader(data_path):
    dataset = []

    with open(data_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line.strip())
            dataset.append({
                "instruction": item["instruction"],  # 질문
                "input": item["input"],              # 문서 내용
                "output": item["output"]             # 정답
            })

    train_data, test_data = train_test_split(dataset, test_size=0.2, shuffle=True, random_state=42)
    return train_data, test_data

In [9]:
train_data, test_data = data_loader("/home/NEUL77/AI-Engineer/data/train/qa_sft_dataset_split.jsonl")

In [10]:
print(len(train_data))
print(len(test_data))

1528
382


## **3. 데이터 포맷 정제**

In [11]:
# RAG 학습을 위한 ChatML 형식 함수
def to_chatml_rag_format(data):
    formatted = []
    for item in data:
        # 시스템 메시지
        prompt_parts = [
            "<|im_start|>system\n당신은 '입찰메이트' 기업의 입찰 전문 컨설턴트 AI입니다. 제공된 문서를 기반으로 질문에 답변합니다.\n<|im_end|>\n",
        ]
        
        # context(input)가 있는 경우
        if item.get("input"):
            prompt_parts.append("<|im_start|>user\n문서:\n" + item["input"].strip() + "\n\n질문:\n" + item["instruction"].strip() + "<|im_end|>\n")
        else:
            prompt_parts.append("<|im_start|>user\n" + item["instruction"].strip() + "<|im_end|>\n")

        # 답변 (output)
        prompt_parts.append("<|im_start|>assistant\n" + item["output"].strip() + "<|im_end|>")

        prompt = "".join(prompt_parts)
        formatted.append({"text": prompt})
    return formatted

# 새로운 함수를 사용하여 학습 데이터 구성
train_chatml = to_chatml_rag_format(train_data)
test_chatml = to_chatml_rag_format(test_data)

In [12]:
# 각 ChatML 문장의 길이를 측정
lengths = [len(tokenizer(item["text"])["input_ids"]) for item in train_chatml]

# 길이 통계 보기
print("< 길이 통계 >")
print(f"→ 평균 길이: {np.mean(lengths):.2f}")
print(f"→ 중앙값: {np.median(lengths)}")
print(f"→ 최대 길이: {np.max(lengths)}")
print(f"→ 상위 95% 길이: {np.percentile(lengths, 95)}")

< 길이 통계 >
→ 평균 길이: 1039.44
→ 중앙값: 958.0
→ 최대 길이: 4783
→ 상위 95% 길이: 1959.8499999999988


In [13]:
dataset_train = Dataset.from_list(train_chatml)
dataset_test = Dataset.from_list(test_chatml)

In [14]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 1528
})

In [15]:
def tokenize(example):
    return tokenizer(
        example["text"],
        padding=True,          # 동적 패딩 → 낭비 최소화
        truncation=True,
        max_length=256         # 길이 분석에 따른 설정
    )

tokenized_train = dataset_train.map(tokenize, batched=True)
tokenized_test = dataset_test.map(tokenize, batched=True)

Map:   0%|          | 0/1528 [00:00<?, ? examples/s]

Map: 100%|██████████| 382/382 [00:00<00:00, 936.94 examples/s]


In [16]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1528
})

## **4. 모델 학습 준비**

In [17]:
# QLoRA 학습 준비
model = prepare_model_for_kbit_training(model)

# LoRA 설정
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

# LoRA 적용
model = get_peft_model(model, lora_config)

## **5. 모델 학습**

In [27]:
pip install pip install -U trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install
Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import TrainingArguments

In [23]:
# 학습 설정
training_args = TrainingArguments(
    output_dir="results_v2/qlora_qwen3_8b_v2",
    logging_dir="results_v2/logs",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    bf16=True,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=False,
    remove_unused_columns=False,
    report_to=None,
)

# SFTTrainer로 학습
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    peft_config=lora_config
)

trainer.train()
trainer.model.save_pretrained("results/qlora_qwen3_8b_v2")
tokenizer.save_pretrained("results/qlora_qwen3_8b_v2")

/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'Qwen/Qwen3-8B' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating eval dataset: 100%|██████████| 382/382 [00:00<00:00, 82948.03 examples/s]
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.737500
20,1.469500
30,1.306600
40,1.231800
50,1.157700
60,1.337100
70,1.261500
80,1.246400
90,1.197000
100,1.186600


KeyboardInterrupt: 

## **6. 모델 병합**

### 6.1 체크포인트 평가

In [ ]:
checkpoints_dir = "results/qlora_qwen3_8b_v2"
checkpoints = sorted([ckpt for ckpt in os.listdir(checkpoints_dir) if ckpt.startswith("checkpoint")])

for ckpt in checkpoints:
    adapter_path = os.path.join(checkpoints_dir, ckpt)
    model_with_adapter = PeftModel.from_pretrained(model, adapter_path)
    model_with_adapter.eval()

    pipe = pipeline("text-generation", model=model_with_adapter, tokenizer=tokenizer)
    result = pipe("국민연금공단의 주요 요구사항은?", max_new_tokens=100)[0]["generated_text"]
    print(f"=== {ckpt} ===\n{result}\n\n")

/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1

=== checkpoint-100 ===
국민연금공단의 주요 요구사항은? 국민연금공단의 주요 요구사항은 무엇인가요? 

국민연금공단은 대한민국의 사회보장제도 중 하나인 국민연금의 운영을 담당하는 공공기관입니다. 따라서 그 주요 요구사항은 국민연금의 **안정적인 운영**, **공정성**, **투명성**, **지속가능성** 등과 같은 원칙을 중심으로 설정됩니다. 구체적으로는 다음과




/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Device set to use cuda:0


=== checkpoint-1000 ===
국민연금공단의 주요 요구사항은? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공단의 주요 요구사항은 무엇인가요? 국민연금공




Device set to use cuda:0


...

=== checkpoint-700 ===
국민연금공단의 주요 요구사항은? 국민연금공단의 주요 요구사항은?

국민연금공단은 대한민국의 사회보장제도 중 하나인 국민연금을 관리하는 기관입니다. 그 주요 요구사항은 다음과 같은 여러 가지로 나눌 수 있습니다:

1. **안정적인 재정 관리**: 국민연금은 장기적인 재정 안정성을 유지하기 위해 지속 가능한 수익률을 달성하고, 향후 연




Device set to use cuda:0


=== checkpoint-800 ===
국민연금공단의 주요 요구사항은? 국민연금공단의 주요 요구사항은 무엇인가요?

국민연금공단은 대한민국의 사회보장제도 중 하나인 국민연금을 운영하는 기관입니다. 따라서 국민연금공단의 주요 요구사항은 다음과 같은 방향으로 설정됩니다:

1. **재정 안정성 확보**: 국민연금은 장기적으로 지속 가능한 재정 구조를 유지해야 하며, 이는 향




Device set to use cuda:0


=== checkpoint-900 ===
국민연금공단의 주요 요구사항은? 국민연금공단의 주요 요구사항은 다음과 같습니다:

1. **정부의 재정 지원**: 국민연금은 현재 정부의 재정 지원을 받고 있으며, 이는 연금 수급자의 생활 안정을 위해 필요합니다. 그러나 정부의 재정 지원이 부족하다는 비판이 있으며, 이는 국민연금의 지속 가능성에 영향을 줄 수 있습니다.

2. **연금 수급자의 생




In [1]:
from model_load import tokenizer, model
from peft import PeftModel

/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [01:33<00:00, 18.75s/it]


### 6.2 모델 병합

In [ ]:
adapter_path = "/home/NEUL77/NEUL/hf/results_v2/qlora_qwen3_8b_v2/"
model = PeftModel.from_pretrained(model, adapter_path)
model = model.merge_and_unload()

model.save_pretrained("/home/NEUL77/NEUL/merged_qwen3_8b_v2")
tokenizer.save_pretrained("/home/NEUL77/NEUL/merged_qwen3_8b_v2")

print("병합 및 저장완료!")

/opt/miniconda/envs/team2_env/lib/python3.11/site-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1

병합 및 저장완료!


### 6.3 모델 성능 확인

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/home/NEUL77/NEUL/merged_qwen3_8b"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)

input_text = "국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘."
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.04s/it]


국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘. 분석을 위한 기초 자료로 활용할 예정이야.
국민연금공단의 이러닝시스템 관련 사업 요구사항을 정리해 주시려면, 해당 시스템의 주요 기능, 요구사항, 운영 환경, 보안 및 개인정보 처리, 법적 규제, 사용자 경험(UX) 등을 포함해야 합니다. 아래는 일반적인 기준을 바탕으로 한 예시입니다. 실제 요구사항은 공단의 공식 문서나 요청서에 명시된 내용을 참고해야 합니다.

---

## **국민연금공단 이학(이러닝) 시스템 사업 요구사항 요약**

### 1. **시스템 개요**
- **시스템명**: 국민연금공단 이러닝 시스템
- **목적**: 직원 및 고객 대상 교육/학습 기능 제공
- **운영 기간**: 5~10년 (계약 기간에 따라 달라짐)
- **운영 환경**: 클라우드 기반, 모바일 및 데스크톱 호환

---

### 2. **주요 기능 요구사항**
#### (1) **학습 관리**
- **코스/과정 관리**: 학습 콘텐츠 생성, 분류, 배치
- **학습 편성**: 사용자별 맞춤형 학습 경로 설정
- **학습 기록 추적**: 학습 시간, 완료 여부, 성적 등 기록
- **학습 성과 분석**: 사용자별 학습 성과 시각화 및 보고

#### (2) **학습 콘텐츠 관리**
- **콘텐츠 제작**: 비디오, 문서, 퀴즈 등 다양한 형식 지원
- **콘텐츠 공유**: 내부/외부 사용자 간 콘텐츠 공유 기능
- **콘텐� 검색**: 키워드, 카테고리, 난이도 등 기반 검색

#### (3) **사용자 관리**
- **사용자 등록/관리**: 직원, 고객 등 다양한 사용자 유형 지원
- **접근 권한 설정**: 역할 기반 접근 제어(RBAC)
- **사용자별 설정**: 학습 기간, 학습 목표 등 설정

#### (4) **모바일 및 멀티디바이스 지원**
- **모바일 호환**: 스마트폰, 태블릿 등 다양한 기기 지원
- **오프라인 학습**: 데이터 저장 및 오프라인 사용 기능

#### (5) **보안 